In [5]:
import os.path
import numpy as np
import pandas as pd
from time import time
from os import listdir
from os.path import isfile, join

In [6]:
def file_csv_list(target_path):
    return [f for f in listdir(target_path) if isfile(join(target_path, f))]

In [7]:
def read_csv(filename):
    # read revise csv file and print cost time
    t0 = time()
    df = pd.read_csv(filename, error_bad_lines=False)
    print("time for read csv: %.2f" % (time()-t0))
    return df

In [8]:
def remove_date_column(df):
    # we don't use VOICE_DATE/VOICE_MONTH bcz it is a monthly data
    if("VOICE_DATE" in df.columns):
        df = df.drop('VOICE_DATE', 1)
        print("Drop VOICE_DATE")
    if("VOICE_MONTH" in df.columns):
        df = df.drop('VOICE_MONTH', 1)
        print("Drop VOICE_MONTH")
    return df

In [9]:
def save_dataframe(df, out_filename):
   # write to csv and no index
    t0 = time()
    df.to_csv(out_filename + ".csv", index=False, encoding='utf-8')
    print("time for output csv file: %.2f" % (time()-t0))

In [11]:
def split_work_holiday(df):
    # let it sort mon ~ sun
    cnt_col_list = df.columns[2:8]
    cnt_col_list = cnt_col_list.append(df.columns[1:2])
    time_col_list = df.columns[9:15]
    time_col_list = time_col_list.append(df.columns[8:9])
    print("cnt_col_list", cnt_col_list)
    print("cnt_col_list[:5]", cnt_col_list[:5])
    print("cnt_col_list[5:]", cnt_col_list[5:])

    print("time_col_list", time_col_list)
    print("time_col_list[:5]",  time_col_list[:5])
    print("time_col_list[5:]",  time_col_list[5:])

    prefix = "VOICE_MT_"
    if "VOICE_MO_HOUR_SUN_CNT" in df.columns:
        prefix = "VOICE_MO_"
 
    # sum
    df[prefix + "workday_cnt"] = 0
    for col in cnt_col_list[:5]:
        df[prefix + "workday_cnt"] += df[col]
#     4 month 4 times workday
    df[prefix + "workday_cnt"] /= 4
    # sum
    df[prefix + "workday_cnt_avg"] = 0
    df[prefix + "workday_cnt_avg"] =df[prefix + "workday_cnt"] / 5
            
    df[prefix + "workday_time"] = 0
    for col in time_col_list[:5]:
        df[prefix + "workday_time"] += df[col]
    #     4 month 4 times workday
    df[prefix + "workday_time"] /= 4
    df[prefix + "workday_time_avg"] = 0
    df[prefix + "workday_time_avg"] =df[prefix + "workday_time"] / 5
    
    df[prefix + "holiday_cnt"] = 0
    for col in cnt_col_list[5:]:
        df[prefix + "holiday_cnt"] += df[col]
    #     4 month 5 times holiday
    df[prefix + "holiday_cnt"] /= 5
    df[prefix + "holiday_cnt_avg"] = 0
    df[prefix + "holiday_cnt_avg"] =df[prefix + "holiday_cnt"] / 2
    
    df[prefix + "holiday_time"] = 0
    for col in time_col_list[5:]:
        df[prefix + "holiday_time"] += df[col]
    #     4 month 5 times holiday
    df[prefix + "holiday_time"] /= 5
    df[prefix + "holiday_time_avg"] = 0
    df[prefix + "holiday_time_avg"] =df[prefix + "holiday_time"] / 2
    return df

In [12]:
# set configure
# path = "../DATA_FULL/"
path = "./CDR_MONTHLY_AGGR/"
# filename = "CDR_MO_HOUR_revise_month_1"
# relative_filename = path + filename + ".csv"
# out_path = "./CDR_MONTHLY_WORKDAY/"
out_path = "./CDR_MONTHLY_AGGR/"

for filename in file_csv_list(path):
    if ".csv" in filename and "WEEK" in filename:
        filename_none_postfix = filename[:-4]
        out_filename =  out_path + filename_none_postfix + "_workday"
        df = read_csv(path + filename)
        df = split_work_holiday(df)
        save_dataframe(df, out_filename)
#         print(df.columns)

time for read csv: 5.11
cnt_col_list Index(['VOICE_MO_HOUR_MON_CNT', 'VOICE_MO_HOUR_TUE_CNT',
       'VOICE_MO_HOUR_WED_CNT', 'VOICE_MO_HOUR_THU_CNT',
       'VOICE_MO_HOUR_FRI_CNT', 'VOICE_MO_HOUR_SAT_CNT',
       'VOICE_MO_HOUR_SUN_CNT'],
      dtype='object')
cnt_col_list[:5] Index(['VOICE_MO_HOUR_MON_CNT', 'VOICE_MO_HOUR_TUE_CNT',
       'VOICE_MO_HOUR_WED_CNT', 'VOICE_MO_HOUR_THU_CNT',
       'VOICE_MO_HOUR_FRI_CNT'],
      dtype='object')
cnt_col_list[5:] Index(['VOICE_MO_HOUR_SAT_CNT', 'VOICE_MO_HOUR_SUN_CNT'], dtype='object')
time_col_list Index(['VOICE_MO_HOUR_MON_TIME', 'VOICE_MO_HOUR_TUE_TIME',
       'VOICE_MO_HOUR_WED_TIME', 'VOICE_MO_HOUR_THU_TIME',
       'VOICE_MO_HOUR_FRI_TIME', 'VOICE_MO_HOUR_SAT_TIME',
       'VOICE_MO_HOUR_SUN_TIME'],
      dtype='object')
time_col_list[:5] Index(['VOICE_MO_HOUR_MON_TIME', 'VOICE_MO_HOUR_TUE_TIME',
       'VOICE_MO_HOUR_WED_TIME', 'VOICE_MO_HOUR_THU_TIME',
       'VOICE_MO_HOUR_FRI_TIME'],
      dtype='object')
time_col_list[5:] Ind

In [13]:
df.head()

,MINING_DW_SUBSCR_NO,VOICE_MT_SUN_CNT,VOICE_MT_MON_CNT,VOICE_MT_TUE_CNT,VOICE_MT_WED_CNT,VOICE_MT_THU_CNT,VOICE_MT_FRI_CNT,VOICE_MT_SAT_CNT,VOICE_MT_SUN_TIME,VOICE_MT_MON_TIME,...,VOICE_MT_FRI_TIME,VOICE_MT_SAT_TIME,VOICE_MT_workday_cnt,VOICE_MT_workday_cnt_avg,VOICE_MT_workday_time,VOICE_MT_workday_time_avg,VOICE_MT_holiday_cnt,VOICE_MT_holiday_cnt_avg,VOICE_MT_holiday_time,VOICE_MT_holiday_time_avg
0,1171,2,1,4,5,8,5,1,71,399,...,119,83,5.75,1.15,355.25,71.05,0.6,0.3,30.8,15.4
1,1179,10,20,10,13,16,17,15,406,1869,...,6031,1433,19.00,3.80,2658.25,531.65,5.0,2.5,367.8,183.9
2,1181,6,4,2,1,9,2,12,234,146,...,116,310,4.50,0.90,157.00,31.40,3.6,1.8,108.8,54.4
3,11121,9,8,4,4,12,4,17,331,281,...,57,438,8.00,1.60,293.00,58.60,5.2,2.6,153.8,76.9
4,11131,8,16,9,5,9,6,10,249,1781,...,189,241,11.25,2.25,682.75,136.55,3.6,1.8,98.0,49.0
